In [6]:
from chxanalys.chx_packages import *
#%matplotlib notebook|
plt.rcParams.update({'figure.max_open_warning': 0})
plt.rcParams.update({ 'image.origin': 'lower'   })
plt.rcParams.update({ 'image.interpolation': 'none'   })
#%reset -f  #for clean up things in the memory
from chxanalys.chx_xpcs_xsvs_jupyter_V1 import run_xpcs_xsvs_single, compress_multi_uids,realtime_xpcs_analysis
import pickle as cpk

import datetime

In [7]:
os.environ['HTTPS_PROXY'] = 'https://proxy:8888'
os.environ['no_proxy'] = 'cs.nsls2.local,localhost,127.0.0.1'

In [8]:
#%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/chx_xpcs_xsvs_jupyter_V1.py

In [9]:
scat_geometry = 'saxs'  #suport 'saxs', 'gi_saxs', 'ang_saxs' (for anisotropics saxs or flow-xpcs)
#scat_geometry = 'gi_saxs'  #suport 'saxs', 'gi_saxs', 'ang_saxs' (for anisotropics saxs or flow-xpcs)
#scat_geometry = 'gi_waxs'  #suport 'saxs', 'gi_saxs', 'ang_saxs' (for anisotropics saxs or flow-xpcs)
                           # gi_waxs define a simple box-shaped ROI 

force_compress = False #True   #force to compress data 
bin_frame = False   #generally make bin_frame as False

para_compress = True    #parallel compress
run_fit_form = False    #run fit form factor 
run_waterfall = False   #run waterfall analysis
run_t_ROI_Inten = True  #run  ROI intensity as a function of time
run_one_time = True  #run  one-time
#run_fit_g2 = True       #run  fit one-time, the default function is "stretched exponential"
fit_g2_func = 'stretched'
run_two_time =   True  #False #True #True #False    #run  two-time
run_four_time = False #True #False   #run  four-time
run_xsvs=  False #False         #run visibility analysis
att_pdf_report = True    #attach the pdf report to CHX olog
qth_interest = 1     #the intested single qth             
use_sqnorm = False    #if True, use sq to normalize intensity
use_imgsum_norm=True  #if True use imgsum to normalize intensity for one-time calculatoin
run_dose = True
pdf_version='_realtime_analysis4'     #for pdf report name
if scat_geometry == 'gi_saxs':run_xsvs= False 
if scat_geometry == 'gi_waxs':run_xsvs= False    
    
  

In [10]:
CYCLE = '2017_1'  
mask_path = '/XF11ID/analysis/2017_1/masks/'
mask_name = 'Apr5_4M_SAXS_mask.npy'

good_start = 5

In [11]:

username =  getpass.getuser()

#username = 'jain'

data_dir0  = create_user_folder(CYCLE, username)
print( data_dir0 )

Results from this analysis will be stashed in the directory /XF11ID/analysis/2017_1/jain/Results/
/XF11ID/analysis/2017_1/jain/Results/


In [6]:
# setup_pargs = {'Ldet': 16035.90385,
# 'center': [1007, 1016],
# 'dpix': 0.075000003562308848,  
# 'lambda_': 1.28481,}

In [7]:
#fp =  '/XF11ID/analysis/2017_1/yuzhang/' + 'Corapal_4M_16m_roi_mask_qval_dict.pkl'
#cpk.dump( [roi_mask,qval_dict],  open(fp, 'wb' ) )

#roi_mask,qval_dict = cpk.load( open(fp,'rb') )

In [12]:
fp = data_dir0 + 'roi_mask_16rings_s2.pkl'

roi_mask,qval_dict = cpk.load( open(fp, 'rb' )  )  #for load the saved roi data

if scat_geometry =='gi_saxs':
    fp = data_dir0 + 'gisaxs_test.pkl'
    qr_map, qz_map, ticks, Qrs, Qzs,  Qr, Qz, inc_x0 = cpk.load( open(fp, 'rb' )  )


In [13]:
run_pargs=  dict(   

    scat_geometry = scat_geometry ,
    force_compress =  force_compress,    #force to compress data 
    para_compress = para_compress,             #parallel compress
    run_fit_form = run_fit_form,             #run fit form factor  
    run_waterfall = run_waterfall,             #run waterfall analysis
    run_t_ROI_Inten = False, # run_t_ROI_Inten,          #run  ROI intensity as a function of time
    run_one_time =  run_one_time,      #run  one-time
    fit_g2_func = fit_g2_func,    
    run_two_time =  run_two_time,    #run  two-time
    run_four_time =  run_four_time,     #run  four-time
    run_xsvs= run_xsvs,            #run visibility analysis
    att_pdf_report = att_pdf_report,       #attach the pdf report to CHX olog
    show_plot = False,                 
    CYCLE =  CYCLE, 
    mask_path =  mask_path, 
    mask_name =    mask_name, 
    good_start   =  good_start,    
    qth_interest =  qth_interest, #the intested single qth,            
    use_sqnorm = use_sqnorm,  #if True, use sq to normalize intensity
    use_imgsum_norm=use_imgsum_norm,#if True use imgsum to normalize intensity for one-time calculatoin
    run_dose = run_dose,
    pdf_version = pdf_version #for pdf report name 
    
)

add_conf = dict(  roi_mask = roi_mask, qval_dict = qval_dict ) 

if scat_geometry == 'gi_saxs':
    add_conf = dict( inc_x0=inc_x0,inc_y0=inc_y0,refl_x0=refl_x0,refl_y0=refl_y0,
                    Qr = Qr,Qz= Qz,qr_map=qr_map,qz_map= qz_map)        
run_pargs.update( add_conf )


In [14]:
roi_mask.shape

(2167, 2070)

In [15]:
#md_update = dict( beam_center_x = 1016,
#                  beam_center_y= 1007,
#                  det_distance= 16035.90385
#                 )
md_update = None

In [12]:
import datetime
START_TIME = time.mktime(  datetime.datetime(2017, 4, 7,  10,  0, 0,0).timetuple() ) 
#STOP_TIME = time.mktime( datetime.datetime(2017, 4,   1,   10,   30, 0,0).timetuple()  )
STOP_TIME = time.mktime(  datetime.datetime(2017, 4, 7,  12, 0 , 0,0).timetuple() ) 

#STOP_TIME = time.mktime( datetime.datetime(2017, 3, 5,  23,  10, 0,0).timetuple() ) 

In [13]:
print( START_TIME, STOP_TIME)

1491278340.0 1491307200.0


In [14]:
%run /XF11ID/analysis/Analysis_Pipelines/Develop/chxanalys/chxanalys/chx_xpcs_xsvs_jupyter_V1.py

In [15]:
#run_xpcs_xsvs_single( uid ='751e0ba8',run_pargs= run_pargs, md_cor = md, return_res= False )

In [ ]:
start_time = START_TIME
stop_time =  START_TIME
#tstep = 10* 60 
while stop_time < STOP_TIME:      
    stop_time =  time.time()
    #print( 'start_time, stop_time:  ')
    #print(  datetime.datetime.fromtimestamp(start_time),
    #        datetime.datetime.fromtimestamp(stop_time ) )          
    realtime_xpcs_analysis( start_time, stop_time,  run_pargs,  md_update = md_update,
                           wait_time = 100,  max_try_num = 72  )  #wait two hour here
    #print('used time: %s'%used_time)
    start_time = stop_time    

Totally 1 uids are found.
**************************************************
Do compress for 6a28ebf0-e24d-4859-a2c7-dc9289d59ed4 now...
Waiting 100 secdons for upcoming data...


# For real-time compress

In [ ]:

if False:
    start_time = START_TIME
    stop_time =  START_TIME
    #tstep = 10* 60
    while stop_time < STOP_TIME:      
        stop_time =  STOP_TIME #time.time()
        #print( 'start_time, stop_time:  ')
        #print(  datetime.datetime.fromtimestamp(start_time),
        #        datetime.datetime.fromtimestamp(stop_time ) )          
        used_time = do_compress_on_line( start_time, stop_time, mask_dict,
                                       wait_time = 6,  max_try_num = 1 ) 
        print('used time: %s'%used_time)
        start_time = stop_time


       
    